# Transaction Prediction Model

This notebook focuses on building a predictive model for transaction counts per customer.

## Approach
- **Problem formulation**: Conditional count modeling (predicting number of transactions)
- **Model**: GLM with Tweedie loss (1 < p < 2) as approximation to Negative Binomial
- **Features**: Time-aware features (recency, frequency, rolling windows)
- **Temporal splitting**: Strict time-based train/test splits to avoid leakage

In [3]:
# Core imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
from pathlib import Path

# Setup: Add project root to Python path for imports
# (Notebooks run from notebooks/ directory, so we need to add parent to path)
PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import project configuration
from src.config import (
    PROJECT_ROOT,
    FIGURES_DIR,
    OUTPUTS_DIR,
    ensure_directories
)

# Ensure directories exist
ensure_directories()

# Use config paths
OUTPUTS = OUTPUTS_DIR
FIGURES = FIGURES_DIR

print("✓ Imports loaded")
print(f"✓ Project root: {PROJECT_ROOT}")

✓ Imports loaded
✓ Project root: /mnt/c/Users/zaido/OneDrive/Bureau/Quod_THA


In [4]:
# Load processed data
from src.data import load_and_process_transactions
import importlib, src.data as data
importlib.reload(data)

df = load_and_process_transactions(force_reprocess=True)

print(f"Loaded {len(df):,} transactions")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Unique customers: {df['customer_id'].nunique():,}")
df.head()

Loading raw transaction data...
Loaded 975,955 raw transactions
Cleaning transaction data...
Removed 676380 duplicate rows (kept first occurrence)
Cleaned data: 299,575 transactions
Saving processed data to /mnt/c/Users/zaido/OneDrive/Bureau/Quod_THA/data/processed/transactions_cleaned.csv
Loaded 299,575 transactions
Date range: 2017-01-01 00:00:00 to 2020-03-17 00:00:00
Unique customers: 2,002


,customer_id,product_id,date
0,9447359,Nissan,2017-01-01
1,1435072,Fiat,2017-01-01
2,5391951,Opel,2017-01-01
3,5391951,Volkswagen,2017-01-01
4,1435072,Peugeot,2017-01-01


## Feature Engineering

Build time-aware features for customer $i$ at cutoff time $t$:

- $N_i(t-1, t)$: Transactions in last 1 month
- $N_i(t-3, t)$: Transactions in last 3 months
- $N_i(t-6, t)$: Transactions in last 6 months
- $N_i(t-3, t) - N_i(t-6, t-3)$: Change in transaction rate
- `days_since_last_tx(i, t)`: Recency feature
- `active_months(i, t)`: Number of active months
- `month_of_year(t)`: Seasonal feature

In [6]:
# Feature Engineering Functions

def compute_transaction_counts(df, customer_id, cutoff_date, window_months):
    """Compute number of transactions for a customer in a time window."""
    # Ensure cutoff_date is pd.Timestamp (tz-naive)
    cutoff_date = pd.Timestamp(cutoff_date)
    if cutoff_date.tz is not None:
        cutoff_date = cutoff_date.tz_localize(None)
    
    # Calculate start date (months before cutoff)
    start_date = cutoff_date - pd.DateOffset(months=window_months)
    
    # df['date'] is datetime64[ns] tz-naive from data.py, compare directly with Timestamp
    mask = (
        (df['customer_id'] == customer_id) &
        (df['date'] >= start_date) &
        (df['date'] < cutoff_date)
    )
    return mask.sum()


def compute_days_since_last_transaction(df, customer_id, cutoff_date):
    """Compute days since last transaction for a customer."""
    # Ensure cutoff_date is pd.Timestamp (tz-naive)
    cutoff_date = pd.Timestamp(cutoff_date)
    if cutoff_date.tz is not None:
        cutoff_date = cutoff_date.tz_localize(None)
    
    # df['date'] is datetime64[ns] tz-naive from data.py, compare directly with Timestamp
    customer_tx = df[df['customer_id'] == customer_id]
    customer_tx_before = customer_tx[customer_tx['date'] < cutoff_date]
    
    if len(customer_tx_before) == 0:
        return np.nan
    
    last_tx_date = pd.Timestamp(customer_tx_before['date'].max())
    delta = (cutoff_date - last_tx_date).days
    return float(delta)


def compute_active_months(df, customer_id, cutoff_date):
    """
    Compute number of months since customer's first transaction.
    
    Returns the number of months between the customer's first ever transaction
    and the cutoff date.
    """
    # Ensure cutoff_date is pd.Timestamp (tz-naive)
    cutoff_date = pd.Timestamp(cutoff_date)
    if cutoff_date.tz is not None:
        cutoff_date = cutoff_date.tz_localize(None)
    
    # df['date'] is datetime64[ns] tz-naive from data.py, compare directly with Timestamp
    customer_tx = df[df['customer_id'] == customer_id]
    customer_tx_before = customer_tx[customer_tx['date'] < cutoff_date]
    
    if len(customer_tx_before) == 0:
        return 0
    
    # Find first transaction date (convert to Timestamp)
    first_tx_date = pd.Timestamp(customer_tx_before['date'].min())
    
    # Calculate number of months between first transaction and cutoff
    # Using year-month periods to count calendar months
    first_period = pd.Period(first_tx_date, freq='M')
    cutoff_period = pd.Period(cutoff_date, freq='M')
    
    # Number of months (inclusive of both first and cutoff month)
    active_months = (cutoff_period - first_period).n + 1
    
    return active_months


def create_feature_vector(df, customer_id, cutoff_date):
    """
    Create feature vector for a customer at a cutoff time.
    
    Features:
    - n_transactions_1m: N_i(t-1, t) - Transactions in last 1 month
    - n_transactions_3m: N_i(t-3, t) - Transactions in last 3 months
    - n_transactions_6m: N_i(t-6, t) - Transactions in last 6 months
    - change_rate_3m: N_i(t-3, t) - N_i(t-6, t-3) - Change in transaction rate
    - days_since_last_tx: Days since last transaction
    - active_months: Number of months since customer's first transaction
    - month_of_year: Month of cutoff date (1-12)
    """
    # Transaction counts in different windows
    n_1m = compute_transaction_counts(df, customer_id, cutoff_date, window_months=1)
    n_3m = compute_transaction_counts(df, customer_id, cutoff_date, window_months=3)
    n_6m = compute_transaction_counts(df, customer_id, cutoff_date, window_months=6)
    
    # Change in transaction rate: N(t-3, t) - N(t-6, t-3)
    # This compares recent 3 months vs previous 3 months
    t_minus_3 = cutoff_date - pd.DateOffset(months=3)
    n_3m_prev = compute_transaction_counts(df, customer_id, t_minus_3, window_months=3)
    change_rate = n_3m - n_3m_prev
    
    # Recency feature
    days_since_last = compute_days_since_last_transaction(df, customer_id, cutoff_date)
    
    # Active months (months since first transaction)
    active_months = compute_active_months(df, customer_id, cutoff_date)
    
    # Seasonal feature (cutoff_date is already pd.Timestamp)
    month_of_year = cutoff_date.month
    
    # Create feature vector
    features = pd.Series({
        'n_transactions_1m': n_1m,
        'n_transactions_3m': n_3m,
        'n_transactions_6m': n_6m,
        'change_rate_3m': change_rate,
        'days_since_last_tx': days_since_last,
        'active_months': active_months,
        'month_of_year': month_of_year,
    })
    
    return features


def create_feature_matrix(df, cutoff_date, customer_ids=None):
    """
    Create feature matrix for all customers (or specified customers) at a single cutoff date.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Transaction dataframe with columns: customer_id, date, product_id
    cutoff_date : pd.Timestamp or str
        Single cutoff date for all customers (features computed using data before this date)
    customer_ids : array-like, optional
        If provided, only compute features for these customers. Otherwise, use all customers
        that have at least one transaction before cutoff_date.
    
    Returns:
    --------
    pd.DataFrame
        Feature matrix with columns: customer_id, cutoff_date, and feature columns.
        Only includes customers with at least one transaction before cutoff_date.
    """
    # Ensure cutoff_date is pd.Timestamp (tz-naive)
    cutoff_date = pd.Timestamp(cutoff_date)
    if cutoff_date.tz is not None:
        cutoff_date = cutoff_date.tz_localize(None)
    
    # df['date'] is datetime64[ns] tz-naive from data.py, compare directly with Timestamp
    # Filter transactions before cutoff
    df_before_cutoff = df[df['date'] < cutoff_date].copy()
    
    # Get customers with at least one transaction before cutoff
    customers_with_history = df_before_cutoff['customer_id'].unique()
    
    # If customer_ids provided, filter to intersection (customers with history AND in provided list)
    if customer_ids is not None:
        customer_ids = pd.Series(customer_ids).unique()
        valid_customers = pd.Series([c for c in customer_ids if c in customers_with_history])
        if len(valid_customers) == 0:
            raise ValueError("No customers in provided list have purchase history before cutoff_date")
    else:
        valid_customers = pd.Series(customers_with_history)
    
    print(f"Computing features for {len(valid_customers):,} customers with purchase history before {cutoff_date}")
    if customer_ids is not None:
        dropped = len(customer_ids) - len(valid_customers)
        if dropped > 0:
            print(f"  (Dropped {dropped} customers without history)")
    
    # Create feature matrix
    feature_rows = []
    for customer_id in valid_customers:
        features = create_feature_vector(df, customer_id, cutoff_date)
        feature_rows.append(features)
    
    feature_df = pd.DataFrame(feature_rows)
    feature_df.insert(0, 'customer_id', valid_customers.values)
    feature_df.insert(1, 'cutoff_date', cutoff_date)
    
    return feature_df


# Test feature engineering on a single customer
test_customer = df['customer_id'].iloc[0]
test_cutoff = pd.Timestamp('2019-01-31')

print(f"Testing feature engineering for customer {test_customer} at {test_cutoff}")
features = create_feature_vector(df, test_customer, test_cutoff)
print("\nFeature vector:")
print(features)

Testing feature engineering for customer 9447359 at 2019-01-31 00:00:00

Feature vector:
n_transactions_1m     11.0
n_transactions_3m     46.0
n_transactions_6m     74.0
change_rate_3m        18.0
days_since_last_tx     6.0
active_months         25.0
month_of_year          1.0
dtype: float64


In [7]:
# Create feature matrix for all customers at a specific cutoff date
# Example: features as of end of January 2019 (for predicting Feb-Apr 2019)
cutoff_date = pd.Timestamp('2019-01-31')

print(f"Creating features at cutoff {cutoff_date}")
print("This may take a moment...")
print("(Customers without purchase history before cutoff will be automatically dropped)\n")

# Create feature matrix - single cutoff date for all customers
# Only customers with at least one transaction before cutoff_date will be included
feature_matrix = create_feature_matrix(df, cutoff_date)

print(f"\nFeature matrix shape: {feature_matrix.shape}")
print("\nFeature summary statistics:")
print(feature_matrix.describe())
print("\nFirst few rows:")
feature_matrix.head(10)

Creating features at cutoff 2019-01-31 00:00:00
This may take a moment...
(Customers without purchase history before cutoff will be automatically dropped)

Computing features for 1,817 customers with purchase history before 2019-01-31 00:00:00

Feature matrix shape: (1817, 9)

Feature summary statistics:
        customer_id          cutoff_date  n_transactions_1m  \
count  1.817000e+03                 1817        1817.000000   
mean   5.451729e+06  2019-01-31 00:00:00           4.751789   
min    1.001614e+06  2019-01-31 00:00:00           0.000000   
25%    3.188310e+06  2019-01-31 00:00:00           0.000000   
50%    5.375641e+06  2019-01-31 00:00:00           0.000000   
75%    7.756617e+06  2019-01-31 00:00:00           3.000000   
max    9.996357e+06  2019-01-31 00:00:00         208.000000   
std    2.623948e+06                  NaN          13.280212   

       n_transactions_3m  n_transactions_6m  change_rate_3m  \
count        1817.000000        1817.000000     1817.000000   


,customer_id,cutoff_date,n_transactions_1m,n_transactions_3m,n_transactions_6m,change_rate_3m,days_since_last_tx,active_months,month_of_year
0,9447359,2019-01-31,11.0,46.0,74.0,18.0,6.0,25.0,1.0
1,1435072,2019-01-31,0.0,0.0,4.0,-4.0,183.0,25.0,1.0
2,5391951,2019-01-31,16.0,44.0,74.0,14.0,9.0,25.0,1.0
3,3944257,2019-01-31,6.0,12.0,28.0,-4.0,3.0,25.0,1.0
4,6223347,2019-01-31,21.0,36.0,63.0,9.0,7.0,25.0,1.0
5,3926262,2019-01-31,39.0,140.0,184.0,96.0,1.0,25.0,1.0
6,9467115,2019-01-31,0.0,136.0,508.0,-236.0,62.0,25.0,1.0
7,4104271,2019-01-31,4.0,8.0,31.0,-15.0,10.0,25.0,1.0
8,4121261,2019-01-31,1.0,1.0,1.0,1.0,22.0,25.0,1.0
9,4275069,2019-01-31,13.0,79.0,219.0,-61.0,7.0,25.0,1.0


In [8]:
# Panel dataset training (multiple monthly cutoffs) + held-out test cutoff
#
# Cutoff rule:
# - Features X_{i,t} use ONLY transactions strictly before the cutoff: date < t
# - Labels   y_{i,t} count transactions in the future horizon:            [t, t + horizon)
#
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_tweedie_deviance, mean_absolute_error, mean_squared_error

POWER = 1.5
HORIZON_MONTHS = 3
TEST_CUTOFF = pd.Timestamp('2019-01-31')


def _norm_ts(x) -> pd.Timestamp:
    x = pd.Timestamp(x)
    if x.tz is not None:
        x = x.tz_localize(None)
    return x.normalize()


def make_future_tx_count(df: pd.DataFrame, cutoff_date: pd.Timestamp, horizon_months: int) -> pd.Series:
    """y_i(t) = #tx for customer i in [t, t+horizon)."""
    cutoff_date = _norm_ts(cutoff_date)
    horizon_end = _norm_ts(cutoff_date + pd.DateOffset(months=horizon_months))
    mask = (df['date'] >= cutoff_date) & (df['date'] < horizon_end)
    y = df.loc[mask].groupby('customer_id').size()
    y.name = 'y_future_tx_count'
    return y


def build_panel_dataset(df: pd.DataFrame, cutoffs, horizon_months: int):
    """Return X, y, keys for a stacked (customer_id, cutoff_date) panel.

    - keys: customer_id, cutoff_date (join-back only)
    - X: numeric features only
    - y: aligned to keys, defaults to 0 if customer has no tx in horizon

    Uniqueness assertion:
    - exactly one row per (customer_id, cutoff_date)
    """
    cutoffs = [_norm_ts(c) for c in cutoffs]

    keys_parts = []
    X_parts = []
    y_parts = []

    for cutoff in cutoffs:
        fm = create_feature_matrix(df, cutoff)

        # exactly one row per (customer_id, cutoff_date)
        assert not fm[['customer_id', 'cutoff_date']].duplicated().any()

        keys = fm[['customer_id', 'cutoff_date']].copy()
        X = fm.drop(columns=['customer_id', 'cutoff_date'])

        # X must be numeric only
        non_numeric = X.select_dtypes(exclude='number').columns.tolist()
        assert non_numeric == [], f"Non-numeric feature columns found: {non_numeric}"

        y_s = make_future_tx_count(df, cutoff, horizon_months)
        y = y_s.reindex(keys['customer_id']).fillna(0).to_numpy(dtype=float)

        keys_parts.append(keys)
        X_parts.append(X)
        y_parts.append(y)

    keys_all = pd.concat(keys_parts, ignore_index=True)
    X_all = pd.concat(X_parts, ignore_index=True)
    y_all = np.concatenate(y_parts, axis=0)

    # Uniqueness + alignment assertions
    assert not keys_all[['customer_id', 'cutoff_date']].duplicated().any(), "Duplicate (customer_id, cutoff_date) rows"
    assert len(X_all) == len(keys_all) == len(y_all)

    return X_all, y_all, keys_all


# --- Time split ---
test_cutoff = _norm_ts(TEST_CUTOFF)
last_train_cutoff = _norm_ts(test_cutoff - pd.DateOffset(months=HORIZON_MONTHS))

# Train cutoffs are all month-ends where cutoff + horizon <= test_cutoff
# We require at least 6 months of history for the 6m features, so we start training
# at the first month-end cutoff >= (min_date + 6 months).
# Pandas >= 3 uses 'ME' for month-end (old alias 'M' was removed)
start_train_cutoff = (df['date'].min() + pd.DateOffset(months=6)).normalize() + pd.offsets.MonthEnd(0)
train_cutoffs = list(pd.date_range(start=start_train_cutoff, end=last_train_cutoff, freq='ME'))
train_cutoffs = [_norm_ts(c) for c in train_cutoffs]

print(f"Test cutoff: {test_cutoff.date()} | Horizon months: {HORIZON_MONTHS}")
print(f"Last train cutoff (cutoff + horizon <= test_cutoff): {last_train_cutoff.date()}")
print(f"# train cutoffs: {len(train_cutoffs)} ({train_cutoffs[0].date()} .. {train_cutoffs[-1].date()})")

X_train, y_train, keys_train = build_panel_dataset(df, train_cutoffs, HORIZON_MONTHS)
X_test, y_test, keys_test = build_panel_dataset(df, [test_cutoff], HORIZON_MONTHS)

print(f"\nTrain panel: X={X_train.shape}, y_mean={y_train.mean():.3f}")
print(f"Test  panel: X={X_test.shape}, y_mean={y_test.mean():.3f}")


Test cutoff: 2019-01-31 | Horizon months: 3
Last train cutoff (cutoff + horizon <= test_cutoff): 2018-10-31
# train cutoffs: 16 (2017-07-31 .. 2018-10-31)
Computing features for 1,412 customers with purchase history before 2017-07-31 00:00:00
Computing features for 1,438 customers with purchase history before 2017-08-31 00:00:00
Computing features for 1,468 customers with purchase history before 2017-09-30 00:00:00
Computing features for 1,514 customers with purchase history before 2017-10-31 00:00:00
Computing features for 1,545 customers with purchase history before 2017-11-30 00:00:00
Computing features for 1,553 customers with purchase history before 2017-12-31 00:00:00
Computing features for 1,573 customers with purchase history before 2018-01-31 00:00:00
Computing features for 1,596 customers with purchase history before 2018-02-28 00:00:00
Computing features for 1,616 customers with purchase history before 2018-03-31 00:00:00
Computing features for 1,639 customers with purchase 

In [ ]:

# --- Tweedie GLM ---
model = TweedieRegressor(power=POWER, link='log', alpha=0.0, max_iter=5000)
model.fit(X_train, y_train)
y_pred = np.clip(model.predict(X_test), 0.0, None)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)

# NOTE: For Tweedie deviance with 1 < power < 2, y_pred must be strictly positive.
# We keep MAE/RMSE on the original predictions, and only floor preds for deviance.
EPS_DEV = 1e-9
y_pred_dev = np.maximum(y_pred, EPS_DEV)
dev = mean_tweedie_deviance(y_test, y_pred_dev, power=POWER)

# Normalized MSE/RMSE (by mean target)
mu = float(np.mean(y_test))
eps = 1e-12
nmse = mse / ((mu ** 2) + eps)
nrmse = rmse / (mu + eps)

# Test target diagnostics
zero_rate = float(np.mean(y_test == 0))
p50 = float(np.quantile(y_test, 0.50))
p95 = float(np.quantile(y_test, 0.95))
print(f"Test target: zero_rate={zero_rate:.3f} | p50={p50:.3f} | p95={p95:.3f}")

print("\n=== TweedieRegressor (held-out test cutoff only) ===")
print(f"Mean Tweedie deviance: {dev:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"NMSE:  {nmse:.4f}")
print(f"NRMSE: {nrmse:.4f}")

# --- Baseline A: y_hat = tx_last_3m = N_i(t-3, t) ---
y_pred_base = X_test['n_transactions_3m'].to_numpy(dtype=float)

mse_b = mean_squared_error(y_test, y_pred_base)
rmse_b = mse_b ** 0.5
mae_b = mean_absolute_error(y_test, y_pred_base)

# Tweedie deviance with 1 < power < 2 requires strictly positive predictions
y_pred_base_dev = np.maximum(y_pred_base, EPS_DEV)
dev_b = mean_tweedie_deviance(y_test, y_pred_base_dev, power=POWER)

nmse_b = mse_b / ((mu ** 2) + eps)
nrmse_b = rmse_b / (mu + eps)

print("\n=== Baseline A (held-out test cutoff only) ===")
print(f"Mean Tweedie deviance: {dev_b:.4f}")
print(f"MAE:  {mae_b:.4f}")
print(f"RMSE: {rmse_b:.4f}")
print(f"NMSE:  {nmse_b:.4f}")
print(f"NRMSE: {nrmse_b:.4f}")
print(f"MAE improvement vs baseline: {(mae_b - mae) / mae_b:.2%}")

Test cutoff: 2019-01-31 | Horizon months: 3
Last train cutoff (cutoff + horizon <= test_cutoff): 2018-10-31
# train cutoffs: 16 (2017-07-31 .. 2018-10-31)
Computing features for 1,412 customers with purchase history before 2017-07-31 00:00:00
Computing features for 1,438 customers with purchase history before 2017-08-31 00:00:00
Computing features for 1,468 customers with purchase history before 2017-09-30 00:00:00
Computing features for 1,514 customers with purchase history before 2017-10-31 00:00:00
Computing features for 1,545 customers with purchase history before 2017-11-30 00:00:00
Computing features for 1,553 customers with purchase history before 2017-12-31 00:00:00
Computing features for 1,573 customers with purchase history before 2018-01-31 00:00:00
Computing features for 1,596 customers with purchase history before 2018-02-28 00:00:00
Computing features for 1,616 customers with purchase history before 2018-03-31 00:00:00
Computing features for 1,639 customers with purchase 

/home/zelmokri/.cache/pypoetry/virtualenvs/quod-tha-fVa_CEgS-py3.11/lib/python3.11/site-packages/sklearn/linear_model/_linear_loss.py:333: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights


In [ ]:
# --- XGBoost (non-linear) with time-based validation + early stopping ---
# We use a single validation cutoff strictly before the test cutoff to tune model capacity
# without ever touching the held-out test cutoff.
# Early stopping controls capacity by selecting the best number of trees.
# Then we retrain on all available pre-test cutoffs using that best number of trees.

import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

VAL_CUTOFF = pd.Timestamp("2018-12-31")

val_cutoff = _norm_ts(VAL_CUTOFF)
test_cutoff = _norm_ts(TEST_CUTOFF)
assert val_cutoff < test_cutoff, "VAL_CUTOFF must be strictly before TEST_CUTOFF"

# Require at least 6 months of history for 6m features
start_train_cutoff = (df["date"].min() + pd.DateOffset(months=6)).normalize() + pd.offsets.MonthEnd(0)

# --- Early-stopping split (train cutoffs must satisfy train_cutoff + horizon <= VAL_CUTOFF) ---
last_es_train_cutoff = _norm_ts(val_cutoff - pd.DateOffset(months=HORIZON_MONTHS))
es_train_cutoffs = list(pd.date_range(start=start_train_cutoff, end=last_es_train_cutoff, freq="ME"))

X_train_es, y_train_es, _ = build_panel_dataset(df, es_train_cutoffs, HORIZON_MONTHS)
X_val, y_val, _ = build_panel_dataset(df, [val_cutoff], HORIZON_MONTHS)

xgb_es = XGBRegressor(
    objective="count:poisson",
    eval_metric="rmse",
    n_estimators=4000,
    max_depth=5,
    n_jobs=-1,
    random_state=42,
)

xgb_es.fit(
    X_train_es,
    y_train_es,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False,
)

best_iteration = int(xgb_es.best_iteration)
best_n_estimators = best_iteration + 1  # best_iteration is 0-based

# --- Final training split (all cutoffs where cutoff + horizon <= TEST_CUTOFF) ---
last_full_train_cutoff = _norm_ts(test_cutoff - pd.DateOffset(months=HORIZON_MONTHS))
full_train_cutoffs = list(pd.date_range(start=start_train_cutoff, end=last_full_train_cutoff, freq="ME"))

X_train_full, y_train_full, _ = build_panel_dataset(df, full_train_cutoffs, HORIZON_MONTHS)
X_test, y_test, _ = build_panel_dataset(df, [test_cutoff], HORIZON_MONTHS)

xgb_final = XGBRegressor(
    objective="count:poisson",
    eval_metric="rmse",
    n_estimators=best_n_estimators,
    max_depth=5,
    n_jobs=-1,
    random_state=42,
)

xgb_final.fit(X_train_full, y_train_full)

y_pred_xgb = np.clip(xgb_final.predict(X_test), 0.0, None)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb) ** 0.5
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print("\n=== XGBoost (time validation + early stopping; evaluated on held-out test cutoff only) ===")
print(f"VAL_CUTOFF: {val_cutoff.date()} | best_iteration={best_iteration} | n_estimators(final)={best_n_estimators}")
print(f"RMSE (test): {rmse_xgb:.4f}")
print(f"MAE  (test): {mae_xgb:.4f}")



=== XGBoostRegressor (held-out test cutoff only) ===
RMSE (eval metric): 18.3196
MAE: 7.2372
